# Data Download and Exploration

In [1]:
!which python

/Users/sam/opt/anaconda3/envs/oy-env/bin/python


This code means that the notebook will re-import your source code in `src` when it is edited (the default is not to re-import, because most modules are assumed not to change over time).  It's a good idea to include it in any exploratory notebook that uses `src` code

In [2]:
%load_ext autoreload
%autoreload 2

This snippet allows the notebook to import from the `src` module.  The directory structure looks like:

```
├── notebooks          <- Jupyter notebooks. Naming convention is a number (for ordering)
│   │                     followed by the topic of the notebook, e.g.
│   │                     01_data_collection_exploration.ipynb
│   └── exploratory    <- Raw, flow-of-consciousness, work-in-progress notebooks
│   └── report         <- Final summary notebook(s)
│
├── src                <- Source code for use in this project
│   ├── data           <- Scripts to download and query data
│   │   ├── sql        <- SQL scripts. Naming convention is a number (for ordering)
│   │   │                 followed by the topic of the script, e.g.
│   │   │                 03_create_pums_2017_table.sql
│   │   ├── data_collection.py
│   │   └── sql_utils.py
```

So we need to go up two "pardir"s (parent directories) to import the `src` code from this notebook.  You'll want to include this code at the top of any notebook that uses the `src` code.

In [3]:
import os
import sys
module_path = os.path.abspath(os.path.join(os.pardir, os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)

The code to download all of the data and load it into a SQL database is in the `data` module within the `src` module.  You'll only need to run `download_data_and_load_into_sql` one time for the duration of the project.

In [4]:
from src.data import data_collection

This line may take as long as 10-20 minutes depending on your network connection and computer specs

In [5]:
data_collection.download_data_and_load_into_sql()

Successfully created database and all tables

Successfully downloaded ZIP file
    https://www2.census.gov/programs-surveys/acs/data/pums/2017/5-Year/csv_pwa.zip
    
Successfully downloaded GZIP file
    https://lehd.ces.census.gov/data/lodes/LODES7/wa/wac/wa_wac_S000_JT00_2017.csv.gz
    
Successfully downloaded GZIP file
    https://lehd.ces.census.gov/data/lodes/LODES7/wa/wa_xwalk.csv.gz
    
Successfully downloaded CSV file
    https://www2.census.gov/geo/docs/maps-data/data/rel/2010_Census_Tract_to_2010_PUMA.txt
    
Successfully loaded CSV file into `pums_2017` table
        
Successfully loaded CSV file into `puma_names_2010` table
        
Successfully loaded CSV file into `wa_jobs_2017` table
        
Successfully loaded CSV file into `wa_geo_xwalk` table
        
Successfully loaded CSV file into `ct_puma_xwalk` table
        


Now it's time to explore the data!

In [6]:
import psycopg2
import pandas as pd

In [7]:
DBNAME = "opportunity_youth"

In [8]:
conn = psycopg2.connect(dbname=DBNAME)

In [9]:
pd.read_sql("SELECT * FROM pums_2017 LIMIT 10;", conn)

,rt,serialno,division,sporder,puma,region,st,adjinc,pwgtp,agep,...,pwgtp71,pwgtp72,pwgtp73,pwgtp74,pwgtp75,pwgtp76,pwgtp77,pwgtp78,pwgtp79,pwgtp80
0,P,2013000000006,9,01,11606,4,53,1061971,27.0,68.0,...,53.0,24.0,39.0,24.0,7.0,27.0,8.0,46.0,25.0,50.0
1,P,2013000000006,9,02,11606,4,53,1061971,22.0,66.0,...,49.0,21.0,38.0,20.0,7.0,25.0,8.0,41.0,22.0,47.0
2,P,2013000000012,9,01,10100,4,53,1061971,22.0,72.0,...,24.0,22.0,25.0,7.0,21.0,35.0,6.0,22.0,6.0,37.0
3,P,2013000000012,9,02,10100,4,53,1061971,19.0,64.0,...,21.0,18.0,19.0,7.0,17.0,29.0,6.0,19.0,6.0,29.0
4,P,2013000000038,9,01,11505,4,53,1061971,4.0,52.0,...,4.0,1.0,2.0,8.0,8.0,1.0,4.0,6.0,1.0,4.0
5,P,2013000000038,9,02,11505,4,53,1061971,4.0,51.0,...,4.0,1.0,1.0,8.0,7.0,1.0,4.0,7.0,2.0,4.0
6,P,2013000000038,9,03,11505,4,53,1061971,7.0,18.0,...,5.0,2.0,3.0,14.0,12.0,2.0,8.0,13.0,3.0,6.0
7,P,2013000000070,9,01,10400,4,53,1061971,15.0,59.0,...,26.0,14.0,14.0,26.0,15.0,16.0,33.0,16.0,15.0,15.0
8,P,2013000000070,9,02,10400,4,53,1061971,18.0,56.0,...,27.0,15.0,16.0,30.0,14.0,18.0,38.0,17.0,17.0,14.0
9,P,2013000000082,9,01,11615,4,53,1061971,90.0,40.0,...,119.0,152.0,33.0,30.0,100.0,77.0,115.0,85.0,114.0,29.0


In [60]:
pd.read_sql("SELECT * FROM puma_names_2010 WHERE puma_name LIKE 'King County%';", conn)

,state_fips,state_name,cpuma0010,puma,geoid,gisjoin,puma_name
0,53,Washington ...,1044,11606,5311606,G53011606,"King County (Northwest)--Shoreline, Kenmore & ..."
1,53,Washington ...,1044,11607,5311607,G53011607,"King County (Northwest)--Redmond, Kirkland Cit..."
2,53,Washington ...,1044,11608,5311608,G53011608,King County (Northwest Central)--Greater Belle...
3,53,Washington ...,1044,11609,5311609,G53011609,"King County (Central)--Sammamish, Issaquah, Me..."
4,53,Washington ...,1044,11610,5311610,G53011610,"King County (Central)--Renton City, Fairwood, ..."
5,53,Washington ...,1044,11613,5311613,G53011613,King County (Southwest Central)--Kent City ...
6,53,Washington ...,1044,11614,5311614,G53011614,King County (Southwest)--Auburn City & Lakelan...
7,53,Washington ...,1044,11615,5311615,G53011615,"King County (Southeast)--Maple Valley, Covingt..."
8,53,Washington ...,1044,11616,5311616,G53011616,"King County (Northeast)--Snoqualmie City, Cott..."
9,53,Washington ...,1045,11611,5311611,G53011611,"King County (West Central)--Burien, SeaTac, Tu..."


In [66]:
pd.read_sql("SELECT * FROM ct_puma_xwalk WHERE puma5ce='11606' OR puma5ce='11607' OR puma5ce='11608' OR puma5ce='11609' OR puma5ce='11610' OR puma5ce='11611' OR puma5ce='11612' OR puma5ce='11613' OR puma5ce='11614' OR puma5ce='11615' OR puma5ce='11616' ;", conn)

,statefp,countyfp,tractce,puma5ce
0,53,033,020100,11606
1,53,033,020200,11606
2,53,033,020300,11606
3,53,033,020401,11606
4,53,033,020402,11606
...,...,...,...,...
261,53,033,032602,11616
262,53,033,032702,11616
263,53,033,032703,11616
264,53,033,032704,11616


In [17]:
pd.read_sql("SELECT * FROM wa_geo_xwalk LIMIT 10;", conn)

,tabblk2010,st,stusps,stname,cty,ctyname,trct,trctname,bgrp,bgrpname,...,stanrcname,necta,nectname,mil,milname,stwib,stwibname,blklatdd,blklondd,createdate
0,530630112024017,53,WA,Washington ...,53063,"Spokane County, WA ...",53063011202,"112.02 (Spokane, WA) ...",530630112024,"4 (Tract 112.02, Spokane, WA) ...",...,None,99999,None,None,None,53000012,12 Spokane WIB ...,47.716671,-117.354964,2019-08-26
1,530630105031024,53,WA,Washington ...,53063,"Spokane County, WA ...",53063010503,"105.03 (Spokane, WA) ...",530630105031,"1 (Tract 105.03, Spokane, WA) ...",...,None,99999,None,None,None,53000012,12 Spokane WIB ...,47.783324,-117.402801,2019-08-26
2,530630101001010,53,WA,Washington ...,53063,"Spokane County, WA ...",53063010100,"101 (Spokane, WA) ...",530630101001,"1 (Tract 101, Spokane, WA) ...",...,None,99999,None,None,None,53000012,12 Spokane WIB ...,48.044817,-117.170360,2019-08-26
3,530630101001014,53,WA,Washington ...,53063,"Spokane County, WA ...",53063010100,"101 (Spokane, WA) ...",530630101001,"1 (Tract 101, Spokane, WA) ...",...,None,99999,None,None,None,53000012,12 Spokane WIB ...,48.015862,-117.164765,2019-08-26
4,530630101001020,53,WA,Washington ...,53063,"Spokane County, WA ...",53063010100,"101 (Spokane, WA) ...",530630101001,"1 (Tract 101, Spokane, WA) ...",...,None,99999,None,None,None,53000012,12 Spokane WIB ...,48.012248,-117.162461,2019-08-26
5,530630113004017,53,WA,Washington ...,53063,"Spokane County, WA ...",53063011300,"113 (Spokane, WA) ...",530630113004,"4 (Tract 113, Spokane, WA) ...",...,None,99999,None,None,None,53000012,12 Spokane WIB ...,47.698614,-117.262875,2019-08-26
6,530630018001012,53,WA,Washington ...,53063,"Spokane County, WA ...",53063001800,"18 (Spokane, WA) ...",530630018001,"1 (Tract 18, Spokane, WA) ...",...,None,99999,None,None,None,53000012,12 Spokane WIB ...,47.679017,-117.373731,2019-08-26
7,530630023003017,53,WA,Washington ...,53063,"Spokane County, WA ...",53063002300,"23 (Spokane, WA) ...",530630023003,"3 (Tract 23, Spokane, WA) ...",...,None,99999,None,None,None,53000012,12 Spokane WIB ...,47.666345,-117.452497,2019-08-26
8,530630018001024,53,WA,Washington ...,53063,"Spokane County, WA ...",53063001800,"18 (Spokane, WA) ...",530630018001,"1 (Tract 18, Spokane, WA) ...",...,None,99999,None,None,None,53000012,12 Spokane WIB ...,47.675200,-117.390052,2019-08-26
9,530630023003025,53,WA,Washington ...,53063,"Spokane County, WA ...",53063002300,"23 (Spokane, WA) ...",530630023003,"3 (Tract 23, Spokane, WA) ...",...,None,99999,None,None,None,53000012,12 Spokane WIB ...,47.665443,-117.449826,2019-08-26


In [18]:
pd.read_sql("SELECT * FROM wa_jobs_2017 LIMIT 10;", conn)

,w_geocode,c000,ca01,ca02,ca03,ce01,ce02,ce03,cns01,cns02,...,cfa02,cfa03,cfa04,cfa05,cfs01,cfs02,cfs03,cfs04,cfs05,createdate
0,530019501001010,1,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2019-08-25
1,530019501001024,1,0,1,0,0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,2019-08-25
2,530019501001026,1,1,0,0,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,2019-08-25
3,530019501001044,1,0,1,0,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,2019-08-25
4,530019501001053,2,0,2,0,0,0,2,0,0,...,0,0,0,0,0,0,0,0,0,2019-08-25
5,530019501001061,7,1,3,3,0,5,2,7,0,...,0,0,0,0,0,0,0,0,0,2019-08-25
6,530019501001090,2,0,2,0,0,2,0,2,0,...,0,0,0,0,0,0,0,0,0,2019-08-25
7,530019501001099,1,1,0,0,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,2019-08-25
8,530019501001104,1,0,0,1,0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,2019-08-25
9,530019501001112,4,1,2,1,0,2,2,4,0,...,0,0,0,0,0,0,0,0,0,2019-08-25


Notice the `LIMIT 10` above.  These tables have a large amount of data in them and **your goal is to use SQL to create your main query, not Pandas**.  Pandas can technically do everything that you need to do, but it will be much slower and more inefficient.  Nevertheless, Pandas is still a useful tool for exploring the data and getting a basic sense of what you're looking at.

In [12]:
list(pd.read_sql("SELECT * FROM pums_2017 LIMIT 10;", conn).columns

['rt',
 'serialno',
 'division',
 'sporder',
 'puma',
 'region',
 'st',
 'adjinc',
 'pwgtp',
 'agep',
 'cit',
 'citwp',
 'cow',
 'ddrs',
 'dear',
 'deye',
 'dout',
 'dphy',
 'drat',
 'dratx',
 'drem',
 'eng',
 'fer',
 'gcl',
 'gcm',
 'gcr',
 'hins1',
 'hins2',
 'hins3',
 'hins4',
 'hins5',
 'hins6',
 'hins7',
 'intp',
 'jwmnp',
 'jwrip',
 'jwtr',
 'lanx',
 'mar',
 'marhd',
 'marhm',
 'marht',
 'marhw',
 'marhyp',
 'mig',
 'mil',
 'mlpa',
 'mlpb',
 'mlpcd',
 'mlpe',
 'mlpfg',
 'mlph',
 'mlpi',
 'mlpj',
 'mlpk',
 'nwab',
 'nwav',
 'nwla',
 'nwlk',
 'nwre',
 'oip',
 'pap',
 'relp',
 'retp',
 'sch',
 'schg',
 'schl',
 'semp',
 'sex',
 'ssip',
 'ssp',
 'wagp',
 'wkhp',
 'wkl',
 'wkw',
 'wrk',
 'yoep',
 'anc',
 'anc1p',
 'anc2p',
 'decade',
 'dis',
 'drivesp',
 'esp',
 'esr',
 'fod1p',
 'fod2p',
 'hicov',
 'hisp',
 'indp',
 'jwap',
 'jwdp',
 'lanp',
 'migpuma',
 'migsp',
 'msp',
 'naicsp',
 'nativity',
 'nop',
 'oc',
 'occp',
 'paoc',
 'pernp',
 'pincp',
 'pobp',
 'povpip',
 'powpuma',
 'pow

In [13]:
pd.read_sql("SELECT * FROM pums_2017;", conn).shape

(359075, 286)

Make sure you close the DB connection when you are done using it

In [ ]:
conn.close()